In [2]:
import tensorflow as tf
import numpy as np
import pickle
import os
from sklearn.metrics import classification_report, accuracy_score

DATASET_PATH = 'dataset'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

full_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

class_names = full_dataset.class_names
num_classes = len(class_names)

dataset_size = len(full_dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)

test_ds = full_dataset.skip(train_size + val_size)

AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 10397 files belonging to 12 classes.


In [3]:
from tensorflow.keras.models import load_model

MODEL_NAME = 'garbage-classifier_final.h5' 
MODEL_PATH = os.path.join('models', MODEL_NAME)

try:
    model = load_model(MODEL_PATH)
except Exception as e:
    print(f"Gagal memuat model: {e}")
    model = None

In [5]:
if model:
    y_pred_probs = model.predict(test_ds)
    
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    y_true = np.concatenate([y for x, y in test_ds], axis=0)
    
    print("\nHasil Metrik Evaluasi")
    
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\n1. Akurasi Keseluruhan: {accuracy * 100:.2f}%\n")
    
    report = classification_report(y_true, y_pred, target_names=class_names)
    print("2, 3, 4. Laporan Presisi, Recall, & F1-Score per Kelas:\n")
    print(report)
else:
    print("Model tidak dimuat, evaluasi dibatalkan.")

33/33 [==============================] - 19s 482ms/step

Hasil Metrik Evaluasi

1. Akurasi Keseluruhan: 90.22%

2, 3, 4. Laporan Presisi, Recall, & F1-Score per Kelas:

              precision    recall  f1-score   support

     battery       0.96      0.95      0.95        94
  biological       0.96      1.00      0.98       104
 brown-glass       0.92      0.90      0.91        72
   cardboard       0.97      0.86      0.91       100
     clothes       0.92      0.98      0.95       111
 green-glass       0.98      0.92      0.95        61
       metal       0.77      0.90      0.83        72
       paper       0.88      0.91      0.89       106
     plastic       0.74      0.70      0.72        88
       shoes       0.94      0.93      0.94       107
       trash       0.95      0.86      0.90        66
 white-glass       0.84      0.85      0.84        72

    accuracy                           0.90      1053
   macro avg       0.90      0.90      0.90      1053
weighted avg       